<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/logo.png?raw=true" alt="2025年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

# ゲノムアセンブル
　近年、シロイヌナズナやイネなどのようなモデル生物だけでなく、さまざまな生物でゲノム配列の整備（ゲノム解読）が進められています。解読されたゲノム配列は、進化学的研究や育種学的研究など、幅広い研究の基盤情報になります。

　次世代シーケンシング（ショートリードシーケンシング）や第三世代シーケンシング（ロングリードシーケンシング）をおこなうと、元々のゲノム配列に由来する、短い塩基配列（**リード、Read**）を多数得られます。ショートリードシーケンシングでは50-300 bpのリードを取得でき、ロングリードシーケンシングでは数kbから数十kb（ときには数百kb~数Mb）のリードを取得できます。ゲノム解読は、それらのリードをつなぎ、元のゲノム配列を復元する試みです。その復元処理の最初のステップは、**ゲノムアセンブル（Genome assemble）**と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/assemble_overview.png?raw=true" height="100px" align="middle">

　　今回の実習では、[約250 kbのゲノム配列](https://raw.githubusercontent.com/CropEvol/lecture/refs/heads/master/textbook_2025/dataset/sample_genome.fa) から仮想的に得たショートリードデータやロングリードデータを使って、それぞれれからからどのようなゲノム配列を復元できるのかを見ていきます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_and_long_reads.png?raw=true" height="100px" align="middle">



### サンプルデータのダウンロード

---



In [ ]:
### サンプルデータのダウンロード
!wget -q -O - https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/illumina_read1.fastq.gz | gzip -dc > illumina_read1.fastq
!wget -q -O - https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/illumina_read2.fastq.gz | gzip -dc > illumina_read2.fastq
!wget -q -O - https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/nanopore.fastq.gz | gzip -dc > nanopore.fastq
!wget -q -O - https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/sample_genome.fa > sample_genome.fa

### ソフトウェアのインストール

In [ ]:
### ソフトウェアのインストール
!wget -q -O 'assemble_softwares.sh' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2025/scripts/assemble_softwares.sh
!bash assemble_softwares.sh

### 今回の勉強内容

1. ショートリードを使ったゲノムアセンブル

1. ロングリードを使ったゲノムアセンブル

1. ゲノム配列を改良する






---
## 1. ショートリードを使ったゲノムアセンブル

　ショートリードのゲノムアセンブルをおこない、ゲノムの再構築を試みてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_read_assemble.png?raw=true" height="100px" align="middle">

### 1-1. ショートリードを確認する
　最初に、ゲノムアセンブルに使用するショートリードデータを確認してみましょう。DNA断片の両端をシーケンシングした**ペアエンド（Paired-end）**という形のリードデータを使用します。ペアエンドのデータは、通常2つのファイルに分かれており、それらを1組のデータとして同時に扱います。

使用するショートリード:
- Read1: /content/illumina_read1.fastq
- Read2: /content/illumina_read2.fastq
- リードの量: ゲノムサイズの約20倍
- リードの長さ: 各80 bp

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/paired_end_reads.png?raw=true" height="200px" align="middle">

#### 復習：リードのデータファイル形式（FASTQ）


　リードのデータは**FASTQ**と呼ばれる形式でファイルに保存されています。FASTQ形式では、4行で1つのリードデータの情報を表しています。

```
@リードの名前
TGAAGCAGT..............(リードの塩基配列)
+
F1GCFCGGE..............(各塩基のクオリティ)
```

- 第1行（@からはじまる行）: リードの名前
- 第2行: リードの塩基配列
- 第3行（+からはじまる行）: 第2行と第4行の区切りのための行
- 第4行: 各塩基配列のクオリティ（ASCIIコードで書かれている）。1文字が何らかのクオリティスコア（数値）を示している。
 - 参考: https://bi.biopapyrus.jp/rnaseq/qc/fastq-quality-score.html



#### seqkitを使ったリードの統計情報確認

 　[seqkit](https://bioinf.shenwei.me/seqkit/)というソフトウェアを使って、ペアエンドのうちRead1（ファイル名: `illumina_read1.fastq`）のリードの数やリードの長さなどの情報を確認してみましょう。

```sh
# リードの統計情報の確認
seqkit stat 塩基配列データ
```

In [ ]:
%%bash
seqkit stat illumina_read1.fastq

　下のコードセルに追記して、もう一方のリード（Read2のファイル名: `illumina_read2.fastq`）の情報も確認してください。

In [ ]:
%%bash
seqkit stat

### 1-2. ゲノムアセンブル

　今回、[SPAdes](https://github.com/ablab/spades)というショートリードのゲノムアセンブルをおこなえるソフトウェアを使って、ショートリードを使ったゲノム配列の復元を試みてみます。

　次のコードセルのプログラムで、ペアエンドショートリードのゲノムアセンブルをおこなってみてください。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/genome_assemble.png?raw=true" height="250px" align="middle">

In [ ]:
%%bash
## ショートリードのアセンブル
## [入力] -1 ショートリードREAD1ファイル -2 ショートリードREAD2ファイル
## [出力] -o 出力フォルダ名
spades.py -1 illumina_read1.fastq -2 illumina_read2.fastq -o spades_output

### 1-3. ショートリードゲノムアセンブルの結果


　アセンブルで得られたゲノム配列（**アセンブリ、Assembly**）を確認してみましょう。Colabサーバー上の次のファイルに保存されています。

/content/spades_output/scaffolds.fasta

　アセンブリは、1本ではなく、複数本の配列に分かれており、元のゲノム配列を完全に再現できていません。アセンブル後の複数本に分かれた配列は **コンティグ（Contig）** と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/contigs.png?raw=true" height="100px" align="middle">


　seqkitを使って、コンティグの配列数や長さの情報をみてみましょう。



In [ ]:
%%bash
## アセンブルの統計情報
## オプション「-a」を追加すると、より多くの情報を得られます
seqkit stat -a spades_output/scaffolds.fasta

#### アセンブルを評価する指標のひとつ「N50」

　再構築したゲノム配列（アセンブリ）を評価する方法をいくつかあります。そのうちのひとつが **N50** と呼ばれる配列の長さを使った指標です。N50は、コンティグを長いものから並べて、全長の50%をカバーするときのコンティグの長さです。ゲノムの半分をその長さ以上のコンティグで再構築できていることを示している指標です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/about_n50.png?raw=true" height="150px" align="middle">


　ショートリードで、元のゲノム配列（約250 kb）のほぼ全長を復元することができました。しかし、143本の配列に分かれており、それぞれの配列の長さも元のゲノム配列からするとまだ短い状態です。

---
## 2. ロングリードを使ったゲノムアセンブル

　ショートリードシーケンシングは、50-300 bpの短い塩基配列を多量に得るシーケンシングです。近年、非常に長い塩基配列を得るシーケンシング技術（**ロングリードシーケンシング、Long read sequencing**）が発達してきました。Oxford Nanopore Technologies社のナノポアシーケンシングや、Pacific Biosciences社のPacBioシーケンシングがそれにあたります。

　現在では、ロングリードを使ったゲノムアセンブルが最もよくおこなわれています。ここでは、ナノポアシーケンシングによるロングリードからどのようなゲノム配列を復元できるかを見ていきましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/long_read_assemble.png?raw=true" height="100px" align="middle">

### 2-1. ロングリードを確認する

　ロングリードのシーケンスデータを見てみましょう。

使用するロングリード:
- ロングリード: /content/nanopore.fastq
- リードの量: ゲノムサイズの約8倍
- リードの長さ: 平均の長さが約5000 bp

　seqkitを使って、ロングリードの統計情報も確認してみましょう。

In [ ]:
%%bash
seqkit stat nanopore.fastq

### 2-2. ゲノムアセンブル
　ロングリードを使ったアセンブルでは、ロングリード用のソフトウェア[Flye](https://github.com/fenderglass/Flye)を使って、ゲノムアセンブルを試みてみましょう。



In [ ]:
%%bash
## ロングリードを使ったアセンブル
flye --scaffold --nano-raw nanopore.fastq --out-dir flye_output

### 2-3. ロングリードゲノムアセンブルの結果


　アセンブリは、次のファイルに保存されます。

/content/flye_output/assembly.fasta

　seqkitで、アセンブリの統計情報を確認してみましょう。

In [ ]:
%%bash
seqkit stat -a flye_output/assembly.fasta

　ロングリードを使ったゲノムアセンブルにより、ショートリードのときの結果よりも全長がやや長くなりました。全長がやや長くなっただけではなく、配列数が大幅に少なくなり、N50の値もかなり向上しました。

|アセンブリ|配列数|全長|N50|
|:--|--:|--:|--:|
|ショートリードのアセンブリ|143|243,324 bp|11,812  bp|
|ロングリードのアセンブリ|8|247,916  bp|56,643  bp|

### 2-4. ショートリードとロングリードのゲノムアセンブル結果を比べる

　元のゲノム配列に対して、ショートリードとロングリードのアセンブリを並べてみて、それぞれのアセンブリの特徴を見てみましょう。ある塩基配列に別の塩基配列を並べる操作を **アライメント（Alignment）** と言います。
- ショートリードアセンブリを元のゲノム配列にアライメントする
- ロングリードアセンブリを元のゲノム配列にアライメントする

In [ ]:
%%bash
## ショートリードアセンブリを元のゲノム配列にアライメント
OUTPUT="ShortReadAssembly"
REF="sample_genome.fa"
CONTIG="spades_output/scaffolds.fasta"
bash Alignment_mm2.sh $REF $OUTPUT $CONTIG

## ロングリードアセンブリを元のゲノム配列にアライメント
OUTPUT="LongReadAssembly"
REF="sample_genome.fa"
CONTIG="flye_output/assembly.fasta"
bash Alignment_mm2.sh $REF $OUTPUT $CONTIG

In [ ]:
## アライメント結果をみる
fa   = "sample_genome.fa"
bam1 = "ShortReadAssembly.bam"
bam2 = "LongReadAssembly.bam"

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": fa, "indexPath": fa+".fai", "id": fa })
aln1 = BamTrack({ "name": bam1, "path": bam1, "height":120, "showCoverage":False })
aln2 = BamTrack({ "name": bam2, "path": bam2, "height":120, "showCoverage":False })
b = igv_notebook.Browser(ref)
b.load_track(aln1)
b.load_track(aln2)


　ショートリードは、長さは短い（今回は150 bp）ものの、塩基の読み間違い（シーケンシングエラー）が少ないリードです。そのため、ショートリードからは、正確性の高いアセンブリを構築できます。

　しかし、生物のゲノムには、同じ塩基パターンの繰り返し（リピート）や重複などを多く含まれています。ショートリードは、リード長よりも長い塩基パターンの繰り返し等に弱く、ショートリードだけでゲノムを再現するのはほぼ不可能です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_reads_and_repeat.png?raw=true" height="250px" align="middle">

　一方で、ロングリードは、シーケンシングエラー率が高く、構築されたコンティグの塩基が正確ではない場合があります。しかし、ショートリードの場合とは異なり、ロングリードを使ったゲノムアセンブリは、非常に長いコンティグを生成できます。そのため、リピート配列を多く含む生物のゲノムアセンブリにおいて、ロングリードは主力的な役割を果たしています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/long_read_assemble2.png?raw=true" height="250px" align="middle">

---
## 3. ゲノム配列を改良する

　リード数が多く、塩基情報が正確なショートリードと、長いコンティグを生成できるロングリードを組み合わせることで、より良いゲノム配列を得ることができます。

　先ほど得られたロングリードのゲノム配列（コンティグ）には、塩基情報が不正確そうな箇所がみられます。ここでは、ショートリードでその不正確性の補正を試みてみましょう。この補正処理は **ポリッシング（Polishing）** と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/polishing.png?raw=true" height="220px" align="middle">

### 3-0. ソフトウェアのインストール

In [ ]:
## 必要なソフトウェアをインストールする
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q -O tools/pilon-1.24.jar https://github.com/broadinstitute/pilon/releases/download/v1.24/pilon-1.24.jar
!wget -q -O Polishing.sh https://github.com/CropEvol/lecture/raw/master/textbook_2025/scripts/Polishing_java.sh

### 3-1. ショートリードを使った補正

In [ ]:
%%bash
CONTIG="flye_output/assembly.fasta"
OUTPUT="assembly"
READ1="illumina_read1.fastq"
READ2="illumina_read2.fastq"

bash Polishing.sh $CONTIG $OUTPUT $READ1 $READ2

### 3-2. 補正後のゲノム配列を確認する

　補正後のゲノム配列にショートリードをアライメントし、その結果をIGVで確認してみましょう。

In [ ]:
%%bash
## 補正前のアセンブリをオリジナルのゲノム配列にアライメント
REF="sample_genome.fa"
OUTPUT1="LongReadAssembly"
CONTIG1="flye_output/assembly.fasta"
bash Alignment_mm2.sh $REF $OUTPUT1 $CONTIG1

## 補正後のアセンブリをオリジナルのゲノム配列にアライメント
OUTPUT2="LongReadAssembly_polished"
CONTIG2="assembly_polished.fasta"
bash Alignment_mm2.sh $REF $OUTPUT2 $CONTIG2

In [ ]:
## アライメント結果をみる
fa   = "sample_genome.fa"
bam1 = "LongReadAssembly.bam"
bam2 = "LongReadAssembly_polished.bam"

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": fa, "indexPath": fa+".fai", "id": fa })
aln1 = BamTrack({ "name": bam1, "path": bam1, "height":120, "showCoverage":False })
aln2 = BamTrack({ "name": bam2, "path": bam2, "height":120, "showCoverage":False })
b = igv_notebook.Browser(ref)
b.load_track(aln1)
b.load_track(aln2)


　今回、ロングリードのアセンブル結果をショートリードで補正しました。そのほかにも、ロングリードをショートリードで補正し、エラーの少ないロングリードにした後にアセンブルをおこなう方法や、ショートリードのアセンブリをロングリードでより長くする方法などがあります。

---
## まとめ

　今回、ショートリードとロングリードを使ってゲノムアセンブルをおこない、ゲノム配列の復元を試みました。

　ショートリードでは、正確なコンティグを得られる一方で、ゲノム全体を再現するほど充分長い配列は得られませんでした。要因としては、ゲノムに含まれるリピート配列の存在などが考えられます。

　ロングリードでは、非常に長いコンティグを得ることができました。しかし、エラー率が高いリードを使ったため、部分的には塩基情報が不正確なコンティグが生成されました。

　ショートリードとロングリードそれぞれの利点を活かすことで、より良いゲノム配列を構築することができます。今回の実習でも、ロングリードで得られた長いコンティグを、ショートリードの正確性で補い、高精度なゲノム配列を得ることができました。このようなロングリードとショートリードを組み合わせたゲノム解読は、現在よく使われています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/major_assemble_proc.png?raw=true" height="220px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

#### ソフトウェアのインストールがうまくいかない場合
　次のコードセルを実行して、別の方法でのインストールを試してみてください。

In [ ]:
### ソフトウェアのインストール
## igv viewer
!pip install -q igv-notebook
!wget -q -O igv_prep.py https://github.com/CropEvol/lecture/raw/master/textbook_2025/scripts/igv_prep.py
## conda-env
!pip install -q condacolab
import condacolab
condacolab.install_miniforge()
## softwares
!conda install -q -y bioconda::seqkit
!conda install -q -y bioconda::spades
!conda install -q -y bioconda::flye
!conda install -q -y bioconda::minimap2
!conda install -q -y bioconda::samtools
!conda install -q -y bioconda::bwa
!conda install -q -y bioconda::pilon
## scripts
!wget -q -O Alignment_mm2.sh https://github.com/CropEvol/lecture/raw/master/textbook_2025/scripts/Alignment_mm2.sh
!wget -q -O Polishing.sh     https://github.com/CropEvol/lecture/raw/master/textbook_2025/scripts/Polishing.sh

#### このテキストで出力される結果ファイル


In [ ]:
!mkdir -p spades_output
!mkdir -p flye_output
!wget -q -O sample_genome.fa.fai https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/Genome_assemble_output/sample_genome.fa.fai
!wget -q -O spades_output/scaffolds.fasta https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/Genome_assemble_output/spades_scaffolds.fasta
!wget -q -O flye_output/assembly.fasta https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/Genome_assemble_output/flye_assembly.fasta
!wget -q -O assembly_polished.fasta https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/Genome_assemble_output/assembly_polished.fasta
!wget -q -O ShortReadAssembly.bam https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/Genome_assemble_output/ShortReadAssembly.bam
!wget -q -O LongReadAssembly.bam https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/Genome_assemble_output/LongReadAssembly.bam
!wget -q -O LongReadAssembly_polished.bam https://github.com/CropEvol/lecture/raw/master/textbook_2025/dataset/Genome_assemble_output/LongReadAssembly_polished.bam